# **Predicting Depression: Machine Learning Challenge**

**In this part we mainly forcus on feature selection , feature creation , feature scalling , model building and evaluation part**

*First of all let's import relevent libraries*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score , precision_score, recall_score

### **Load and Process the Dataset**

In [15]:
df_student = pd.read_csv('../data/processed_data/train_student.csv')
df_test_student = pd.read_csv('../data/processed_data/test_student.csv')

df_Wf = pd.read_csv('../data/processed_data/train_Wf.csv')
df_test_Wf = pd.read_csv('../data/processed_data/test_Wf.csv')

In [16]:
X_s = df_student.drop(columns=['Depression']) 
y_s = df_student['Depression']

X_w = df_Wf.drop(columns=['Depression'])
y_w = df_Wf['Depression']

x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s, test_size=0.2, random_state=0, stratify=y_s)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_w, y_w, test_size=0.2, random_state=0, stratify=y_w)  


In [17]:
ds= df_test_student[['id']]  
dw= df_test_Wf[['id']]

# Find common columns (excluding ID)
common_cols_s = set(x_train_s.columns) & set(df_test_student.columns) - {'id'}
common_cols_w = set(X_train_w.columns) & set(df_test_Wf.columns) - {'id'}

# Keep only common columns
x_train_s = x_train_s[list(common_cols_s)]
x_test_s = x_test_s[list(common_cols_s)]
df_test_student = df_test_student[list(common_cols_s)]

x_train_w = X_train_w[list(common_cols_w)]
x_test_w = X_test_w[list(common_cols_w)]
df_test_Wf = df_test_Wf[list(common_cols_w)]

### **Remove Constant, Quasi Constant and Duplicated Values**

In [18]:
const_filter_s = VarianceThreshold(threshold=0)
const_filter_s.fit(x_train_s)
x_train_sc = const_filter_s.transform(x_train_s)
x_test_sc = const_filter_s.transform(x_test_s)

df_test_sc = const_filter_s.transform(df_test_student)

const_filter_w = VarianceThreshold(threshold=0)
const_filter_w.fit(x_train_w)
x_train_wc = const_filter_w.transform(x_train_w)
x_test_wc = const_filter_w.transform(x_test_w)

df_test_wc = const_filter_w.transform(df_test_Wf)

In [6]:
const_filter_s = VarianceThreshold(threshold=0.00002)
const_filter_s.fit(x_train_sc)
x_train_sq = const_filter_s.transform(x_train_sc)
x_test_sq = const_filter_s.transform(x_test_sc)

df_test_sq = const_filter_s.transform(df_test_sc)

const_filter_w = VarianceThreshold(threshold=0.00005)
const_filter_w.fit(x_train_wc)
x_train_wq = const_filter_w.transform(x_train_wc)
x_test_wq = const_filter_w.transform(x_test_wc)

df_test_wq = const_filter_w.transform(df_test_wc)

***No any duplicate values in the dataset then we don't remove duplicate Data***

### **Feature Scalling**

In [19]:

scaler1 = StandardScaler()
X_train_ss = scaler1.fit_transform(x_train_sq)
X_test_ss = scaler1.transform(x_test_sq)

df_test_ss = scaler1.transform(df_test_sq)


scaler2 = StandardScaler()
X_train_ws = scaler2.fit_transform(x_train_wq)
X_test_ws = scaler2.transform(x_test_wq)

df_test_ws = scaler2.transform(df_test_wq)


### **Linear Regression**

In [20]:
from sklearn.metrics import accuracy_score, f1_score , precision_score, recall_score

def traning_scores(y_act,y_pred):
    acc = accuracy_score(y_act,y_pred)
    f1 = f1_score(y_act,y_pred)
    pre = precision_score(y_act,y_pred)
    rec = recall_score(y_act,y_pred)
    print(f'Traning Scores\nAccuracy: {acc} \nF1 Score: {f1} \nPrecision: {pre} \nRecall: {rec}')

def validation_scores(y_act,y_pred):
    acc = accuracy_score(y_act,y_pred)
    f1 = f1_score(y_act,y_pred)
    pre = precision_score(y_act,y_pred)
    rec = recall_score(y_act,y_pred)
    print(f'Testing Scores\nAccuracy: {acc} \nF1 Score: {f1} \nPrecision: {pre} \nRecall: {rec}')

**Model For Working Frofettional**

In [21]:
lr_w = LogisticRegression(solver='saga', n_jobs=-1,C= 0.01, class_weight = None, fit_intercept= True, l1_ratio= 0.3, max_iter= 500, penalty= 'elasticnet', tol= 0.01)

lr_w.fit(X_train_ws, y_train_w)

y_train_pred = lr_w.predict(X_train_ws)
traning_scores(y_train_w,y_train_pred)

y_test_pred = lr_w.predict(X_test_ws)
validation_scores(y_test_w,y_test_pred)

Traning Scores
Accuracy: 0.9623756557323524 
F1 Score: 0.7475721066082512 
Precision: 0.8326284970722186 
Recall: 0.6782827613621306
Testing Scores
Accuracy: 0.9626485568760611 
F1 Score: 0.7504363001745201 
Precision: 0.8317214700193424 
Recall: 0.6836248012718601


**Model for Student**

In [ ]:
lr_s = LogisticRegression(solver='saga', n_jobs=-1,C= 0.01, class_weight = None, fit_intercept= True, l1_ratio= 0.3, max_iter= 500, penalty= 'elasticnet', tol= 0.01)
lr_s.fit(X_train_ss, y_train_s)

y_train_pred = lr_s.predict(X_train_ss)
traning_scores(y_train_s,y_train_pred)

y_test_pred = lr_s.predict(X_test_ss)
validation_scores(y_test_s,y_test_pred)

Traning Scores
Accuracy: 0.852815568176815 
F1 Score: 0.8760336707828086 
Precision: 0.8622527191765822 
Recall: 0.8902622851974676
Testing Scores
Accuracy: 0.8517344602923049 
F1 Score: 0.8752592592592593 
Precision: 0.8607226107226107 
Recall: 0.890295358649789


***For Lodistic regression Working profettional Data work well but Student data has low prediction accuracy***

### **Naïve Bayes**

**Model for WF**

In [24]:
nb_w = GaussianNB()
nb_w.fit(X_train_ws, y_train_w)

y_train_pred = nb_w.predict(X_train_ws)
traning_scores(y_train_w, y_train_pred)

y_test_pred = nb_w.predict(X_test_ws)
validation_scores(y_test_w, y_test_pred)

Traning Scores
Accuracy: 0.7784985089571407 
F1 Score: 0.40578102189781023 
Precision: 0.2602329326292926 
Recall: 0.9207632171723864
Testing Scores
Accuracy: 0.7787209960384833 
F1 Score: 0.4057056003741377 
Precision: 0.26027602760276025 
Recall: 0.9194488606253313


**Model for Student**

In [25]:
nb_s = GaussianNB()
nb_s.fit(X_train_ss, y_train_s)

y_train_pred = nb_s.predict(X_train_ss)
traning_scores(y_train_s, y_train_pred)

y_test_pred = nb_s.predict(X_test_ss)
validation_scores(y_test_s, y_test_pred)

Traning Scores
Accuracy: 0.4883987143926386 
F1 Score: 0.2268795741849634 
Precision: 0.9676503972758229 
Recall: 0.12850467289719625
Testing Scores
Accuracy: 0.48265539707695015 
F1 Score: 0.2119098712446352 
Precision: 0.9634146341463414 
Recall: 0.11904761904761904


***Both two Datasets Naive base does not perfome well***

### **Decission Tree**

In [26]:
dt_w = DecisionTreeClassifier(random_state=42)
dt_w.fit(X_train_ws, y_train_w)

y_train_pred = dt_w.predict(X_train_ws)
traning_scores(y_train_w, y_train_pred)

y_test_pred = dt_w.predict(X_test_ws)
validation_scores(y_test_w, y_test_pred)

Traning Scores
Accuracy: 1.0 
F1 Score: 1.0 
Precision: 1.0 
Recall: 1.0
Testing Scores
Accuracy: 0.9377040616429411 
F1 Score: 0.6219286657859974 
Precision: 0.6201264488935722 
Recall: 0.6237413884472708


In [27]:
dt_s = DecisionTreeClassifier(random_state=42)
dt_s.fit(X_train_ss, y_train_s)

y_train_pred = dt_s.predict(X_train_ss)
traning_scores(y_train_s, y_train_pred)

y_test_pred = dt_s.predict(X_test_ss)
validation_scores(y_test_s, y_test_pred)

Traning Scores
Accuracy: 1.0 
F1 Score: 1.0 
Precision: 1.0 
Recall: 1.0
Testing Scores
Accuracy: 0.7754886423666139 
F1 Score: 0.8073727149116181 
Precision: 0.8094516813086944 
Recall: 0.805304400241109


### **Random Forest**

In [28]:
rf_w = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_w.fit(X_train_ws, y_train_w)

y_train_pred = rf_w.predict(X_train_ws)
traning_scores(y_train_w, y_train_pred)

y_test_pred = rf_w.predict(X_test_ws)
validation_scores(y_test_w, y_test_pred)

Traning Scores
Accuracy: 0.9999782329509589 
F1 Score: 0.9998674794593162 
Precision: 1.0 
Recall: 0.9997349940373659
Testing Scores
Accuracy: 0.9556397196465108 
F1 Score: 0.6709719082983533 
Precision: 0.8586776859504133 
Recall: 0.5506094329623742


In [29]:
rf_s = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_s.fit(X_train_ss, y_train_s)

y_train_pred = rf_s.predict(X_train_ss)
traning_scores(y_train_s, y_train_pred)

y_test_pred = rf_s.predict(X_test_ss)
validation_scores(y_test_s, y_test_pred)


Traning Scores
Accuracy: 1.0 
F1 Score: 1.0 
Precision: 1.0 
Recall: 1.0
Testing Scores
Accuracy: 0.8425779186476492 
F1 Score: 0.867712340929269 
Precision: 0.8523255813953489 
Recall: 0.8836648583484027


***Random Forest is good but not as Logistic regression***

### **XGBoost Classifier**

In [30]:
xgb_w = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_w.fit(X_train_ws, y_train_w)

y_train_pred = xgb_w.predict(X_train_ws)
traning_scores(y_train_w, y_train_pred)

y_test_pred = xgb_w.predict(X_test_ws)
validation_scores(y_test_w, y_test_pred)


c:\Users\npafe\Desktop\Machine Learning\Depression Prediction\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:27:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Traning Scores
Accuracy: 0.969471713719771 
F1 Score: 0.8001140169600228 
Precision: 0.8655565834104224 
Recall: 0.743871737114085
Testing Scores
Accuracy: 0.9602977667493796 
F1 Score: 0.7394285714285714 
Precision: 0.8022318660880348 
Recall: 0.6857445680975093


In [31]:
xgb_s = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_s.fit(X_train_ss, y_train_s)

y_train_pred = xgb_s.predict(X_train_ss)
traning_scores(y_train_s, y_train_pred)

y_test_pred = xgb_s.predict(X_test_ss)
validation_scores(y_test_s, y_test_pred)


c:\Users\npafe\Desktop\Machine Learning\Depression Prediction\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:27:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Traning Scores
Accuracy: 0.9139699731431339 
F1 Score: 0.9272361659343115 
Precision: 0.9163845134697483 
Recall: 0.9383479047331926
Testing Scores
Accuracy: 0.8376474731466808 
F1 Score: 0.8625521765056648 
Precision: 0.8533923303834808 
Recall: 0.8719107896323086


***XGBoost Clssifier also works well for the dataset***

***Now let's assemble multiple model to get best accurate predictions***

### **Hyperparameter Tuning**

In [ ]:
def objective(trial):
    C = trial.suggest_loguniform('C', 0.0001, 10)  # Regularization strength
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 0.9)  # ElasticNet mix ratio
    max_iter = trial.suggest_int('max_iter', 100, 1000, step=100)  # Iterations
    tol = trial.suggest_loguniform('tol', 0.0001, 0.1)  # Tolerance

    # Define the model
    model = LogisticRegression(solver='saga', n_jobs=-1, penalty='elasticnet',
                               fit_intercept=True, C=C, l1_ratio=l1_ratio,
                               max_iter=max_iter, tol=tol)

    # Perform cross-validation
    accuracy = cross_val_score(model, X_train_ws, y_train_w, cv=5, scoring='accuracy', n_jobs=-1).mean()
    
    return accuracy  # Maximize accuracy

# Create Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get best hyperparameters
best_params = study.best_params
print("Best Hyperparameters (Optuna):", best_params)

# Train model with best hyperparameters
best_model = LogisticRegression(solver='saga', n_jobs=-1, penalty='elasticnet', fit_intercept=True, **best_params) # change Model and grid for others 
best_model.fit(X_train_ws, y_train_w)

# Train and evaluate the best model
y_train_pred = best_model.predict(X_train_ws)
traning_scores(y_train_w, y_train_pred)

y_test_pred = best_model.predict(X_test_ws)
validation_scores(y_test_w, y_test_pred)

### **Model Assemble**

In [32]:


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Define individual models
lr = LogisticRegression(solver='saga', n_jobs=-1,C= 0.01, class_weight = None, fit_intercept= True, l1_ratio= 0.3, max_iter= 500, penalty= 'elasticnet', tol= 0.01, random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42, n_jobs=-1)

# Create a Voting Classifier (Soft Voting for better probability estimates)
voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('lgbm', lgbm)],
    voting='soft', 
    weights=[2.9853169041728673, 0.5770399926666891, 0.5075508446786692], # 'hard' for majority voting, 'soft' for probability averaging
    n_jobs=-1
)

# Train the ensemble model
voting_clf.fit(X_train_ws, y_train_w)

# Predict on training data
y_train_pred = voting_clf.predict(X_train_ws)
print("Training Scores:")
traning_scores(y_train_w, y_train_pred)

# Predict on test data
y_test_pred = voting_clf.predict(X_test_ws)
print("Validation Scores:")
validation_scores(y_test_w, y_test_pred)


Training Scores:
Traning Scores
Accuracy: 0.9633987070372869 
F1 Score: 0.7493478422896326 
Precision: 0.8563884156729131 
Recall: 0.6660924870809594
Validation Scores:
Testing Scores
Accuracy: 0.9629532889295198 
F1 Score: 0.7468015471585837 
Precision: 0.8514246947082768 
Recall: 0.6650768415474297


**Parameter Tuning For Best weights**

In [ ]:


import optuna
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define individual models with new names
log_reg = LogisticRegression(solver='saga', n_jobs=-1, C=0.01, class_weight=None, fit_intercept=True, 
                             l1_ratio=0.3, max_iter=500, penalty='elasticnet', tol=0.01, random_state=42)
rand_forest = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
light_gbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42, n_jobs=-1)

# Define objective function for optimization
def objective(trial):
    w1 = trial.suggest_float("w1", 0.5, 3.0)  # Logistic Regression weight
    w2 = trial.suggest_float("w2", 0.5, 3.0)  # Random Forest weight
    w3 = trial.suggest_float("w3", 0.5, 3.0)  # LightGBM weight

    ensemble_clf = VotingClassifier(
        estimators=[('log_reg', log_reg), ('rand_forest', rand_forest), ('light_gbm', light_gbm)],
        voting='soft',
        weights=[w1, w2, w3],
        n_jobs=-1
    )

    # Train model
    ensemble_clf.fit(X_train_ws, y_train_w)
    
    # Evaluate on validation set
    y_val_pred = ensemble_clf.predict(X_test_ws)
    accuracy = accuracy_score(y_test_w, y_val_pred)
    
    return accuracy  # Maximizing accuracy

# Run Optuna optimization
study = optuna.create_study(direction="maximize")  # Maximize accuracy
study.optimize(objective, n_trials=30)  # Try 30 different weight combinations

# Get best weights
best_weights = [study.best_params["w1"], study.best_params["w2"], study.best_params["w3"]]
print(f"Best Weights Found: {best_weights}")

# Train final model with best weights
best_ensemble_clf = VotingClassifier(
    estimators=[('log_reg', log_reg), ('rand_forest', rand_forest), ('light_gbm', light_gbm)],
    voting='soft',
    weights=best_weights,  #Best Weights Found: [2.9853169041728673, 0.5770399926666891, 0.5075508446786692]
    n_jobs=-1
)
best_ensemble_clf.fit(X_train_ws, y_train_w)

# Predict on training data
y_train_pred = best_ensemble_clf.predict(X_train_ws)
print("Training Scores:")
traning_scores(y_train_w, y_train_pred)

# Predict on test data
y_test_pred = best_ensemble_clf.predict(X_test_ws)
print("Validation Scores:")
validation_scores(y_test_w, y_test_pred)



```Python

Best Weights Found: [2.9640932516597274, 0.5310683971855825, 2.8126479186454434]

Training Scores:
Traning Scores
Accuracy: 0.9642585054744128 
F1 Score: 0.7565243179122183 
Precision: 0.8587779835044606 
Recall: 0.6760302106797403

Validation Scores:
Testing Scores
Accuracy: 0.9624744242740847 
F1 Score: 0.7446682464454977 
Precision: 0.8441907320349228 
Recall: 0.6661367249602543
```

In [33]:


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Define individual models
lr = LogisticRegression(solver='saga', n_jobs=-1,C= 0.01, class_weight = None, fit_intercept= True, l1_ratio= 0.3, max_iter= 500, penalty= 'elasticnet', tol= 0.01, random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42, n_jobs=-1)

# Create a Voting Classifier (Soft Voting for better probability estimates)
best_voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('lgbm', lgbm)],
    voting='soft',
    weights=[1.9432688188611906, 0.8690600198005576, 1.2477096350059103], 
    n_jobs=-1
)
best_voting_clf.fit(X_train_ss, y_train_s)

# Predict on training data
y_train_pred = best_voting_clf.predict(X_train_ss)
print("Training Scores:")
traning_scores(y_train_s, y_train_pred)

# Predict on test data
y_test_pred = best_voting_clf.predict(X_test_ss)
print("Validation Scores:")
validation_scores(y_test_s, y_test_pred)


Training Scores:
Traning Scores
Accuracy: 0.8721437062475235 
F1 Score: 0.892468340368807 
Precision: 0.8772019216771 
Recall: 0.9082755501959602
Validation Scores:
Testing Scores
Accuracy: 0.8496214122204614 
F1 Score: 0.873293768545994 
Precision: 0.8600233781414378 
Recall: 0.8869801084990958


**Parameter Tuning for Best Weights**

In [ ]:
import optuna
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define individual models
lr = LogisticRegression(solver='saga', n_jobs=-1, C=0.01, class_weight=None, fit_intercept=True, 
                        l1_ratio=0.3, max_iter=500, penalty='elasticnet', tol=0.01, random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42, n_jobs=-1)

# Define objective function for optimization
def objective(trial):
    w1 = trial.suggest_float("w1", 0.5, 3.0)  # Logistic Regression weight
    w2 = trial.suggest_float("w2", 0.5, 3.0)  # Random Forest weight
    w3 = trial.suggest_float("w3", 0.5, 3.0)  # LightGBM weight

    voting_clf = VotingClassifier(
        estimators=[('lr', lr), ('rf', rf), ('lgbm', lgbm)],
        voting='soft',
        weights=[w1, w2, w3],
        n_jobs=-1
    )

    # Train model
    voting_clf.fit(X_train_ss, y_train_s)
    
    # Evaluate on validation set
    y_val_pred = voting_clf.predict(X_test_ss)
    accuracy = accuracy_score(y_test_s, y_val_pred)
    
    return accuracy  # Maximizing accuracy

# Run Optuna optimization
study = optuna.create_study(direction="maximize")  # Maximize accuracy
study.optimize(objective, n_trials=50)  # Try 30 different weight combinations

# Get best weights
best_weights = [study.best_params["w1"], study.best_params["w2"], study.best_params["w3"]]
print(f"Best Weights Found: {best_weights}")

# Train final model with best weights
best_voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('lgbm', lgbm)],
    voting='soft',
    weights=best_weights,
    n_jobs=-1
)
best_voting_clf.fit(X_train_ss, y_train_s)

# Predict on training data
y_train_pred = best_voting_clf.predict(X_train_ss)
print("Training Scores:")
traning_scores(y_train_s, y_train_pred)

# Predict on test data
y_test_pred = best_voting_clf.predict(X_test_ss)
print("Validation Scores:")
validation_scores(y_test_s, y_test_pred)


```Python
Best Weights Found: [1.9432688188611906, 0.8690600198005576, 1.2477096350059103]

Training Scores:
Traning Scores
Accuracy: 0.8717914850526131 
F1 Score: 0.8922359558877951 
Precision: 0.8764722989675731 
Recall: 0.9085770274344287

Validation Scores:
Testing Scores
Accuracy: 0.8506779362563832 
F1 Score: 0.8741839762611276 
Precision: 0.8609000584453536 
Recall: 0.8878842676311031
```

### **Save the Predictions**

In [34]:
y_pred_student = lr_s.predict(df_test_ss)
y_pred_wf = lr_w.predict(df_test_ws)

In [35]:
ds['Depression'] = y_pred_student
dw['Depression'] = y_pred_wf

df_final = pd.concat([dw,ds])  # Merge predictions
df_final = df_final.sort_values(by='id')  # Sort to match original order

df_final.to_csv('../predictions/lr_5.csv', index=False)  # Save to file
